In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

import pandas as pd
#import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
SELECT *,concat(d.forename,' ',d.surname) as driver
FROM PLAYGROUND.DATATHON.RESULTS rs
left join playground.datathon.RACES rc on rs.raceid = rc.raceid
--left join playground.datathon.status s on rs.statusid = s.statusid
left join playground.datathon.constructors c on rs.constructorid = c.constructorid
left join playground.datathon.drivers d on rs.driverid = d.driverid
left join playground.datathon.qualifying q on q.raceid = rc.raceid and q.driverid = rs.driverid
where 
--rc.year = '2024' and 
c.name in ('Mercedes')


In [ ]:
#Exploratory Analysis

df=results.to_pandas()

def time_to_milliseconds(time_str):
    if pd.isnull(time_str):  # Handles NaN
        return None
    if isinstance(time_str, str):
        try:
            minutes, seconds = time_str.split(':')
            total_ms = (int(minutes) * 60 + float(seconds)) * 1000
            return int(total_ms)
        except ValueError:
            return None  # If split fails or conversion fails
    return None  # Not a string

df['q3_time']=df['Q3'].apply(time_to_milliseconds)
df= df.dropna(subset=['POSITION'])

#df['q3_time']=pd.to_numeric(df['Q3'], errors='coerce')
df['grid']=df['GRID'].astype(int)
df['finish']=df['POSITION'].astype(int)
df['qual_dif']=df['grid']-df['finish']
df['points']=df['POINTS'].astype(float)

summary = df[['DRIVER','GRID','finish','q3_time','points']].describe()
print(summary)

In [ ]:
#Coorelation and predictive model

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

# Prepare dataset (drop NAs)


def time_to_milliseconds(time_str):
    if pd.isnull(time_str):  # Handles NaN
        return None
    if isinstance(time_str, str):
        try:
            minutes, seconds = time_str.split(':')
            total_ms = (int(minutes) * 60 + float(seconds)) * 1000
            return int(total_ms)
        except ValueError:
            return None  # If split fails or conversion fails
    return None  # Not a string
    
df=results.to_pandas()

df= df.dropna(subset=['POSITION'])
df['q3_time']=df['Q3'].apply(time_to_milliseconds)
df['grid']=df['GRID'].astype(int)
df['finish']=df['POSITION'].astype(int)



df = df.dropna(subset=['q3_time', 'grid', 'finish'])
X = df[['q3_time', 'grid']]
y = df['finish']



# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate and predict
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2:.2f}")

#Mean absolute error
print("MAE : ", mean_absolute_error(y_test,y_pred))

